<a href="https://colab.research.google.com/github/ramadhantriyansyah-design/violajones_rama/blob/main/Viola_Jones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# ============================================================
#                 INSTALL & IMPORT LIBRARIES
# ============================================================

!pip install opencv-python opencv-contrib-python

from google.colab import drive
drive.mount('/content/drive')

from google.colab import output
from base64 import b64decode
from IPython.display import HTML, display
from PIL import Image as PILImage
from io import BytesIO
import numpy as np
import cv2
import os

# ============================================================
#          SETUP FOLDER PENYIMPANAN DI GOOGLE DRIVE
# ============================================================

DRIVE_FOLDER = "/content/drive/MyDrive/Face_Project/"
DATASET_DIR = DRIVE_FOLDER + "dataset_faces/"
MODEL_PATH = DRIVE_FOLDER + "face_model.yml"

os.makedirs(DATASET_DIR, exist_ok=True)

print("✔ Folder Drive OK:", DRIVE_FOLDER)

# ============================================================
#                HAAR CASCADE (bawaan OpenCV)
# ============================================================

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade  = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
smile_cascade= cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_smile.xml")

# ============================================================
#                  DATASET + RECOGNIZER
# ============================================================

recognizer = cv2.face.LBPHFaceRecognizer_create()

if os.path.exists(MODEL_PATH):
    recognizer.read(MODEL_PATH)
    print("✔ Model berhasil dimuat dari Google Drive:", MODEL_PATH)
else:
    print("⚠ Belum ada model, akan dilatih otomatis setelah menyimpan wajah.")

# ============================================================
#                 TRAINING OTOMATIS
# ============================================================

def auto_train():
    images = []
    labels = []

    files = os.listdir(DATASET_DIR)
    if len(files) == 0:
        print("⚠ Dataset kosong, training dibatalkan.")
        return

    for fname in files:
        path = os.path.join(DATASET_DIR, fname)
        img = PILImage.open(path).convert("L")
        img_np = np.array(img, "uint8")

        label = int(fname.split("_")[1].split(".")[0])
        images.append(img_np)
        labels.append(label)

    recognizer.train(images, np.array(labels))
    recognizer.write(MODEL_PATH)
    print("✅ Training otomatis selesai & model disimpan ke Google Drive.")

# ============================================================
#               FUNGSI DETEKSI + RECOGNIZE
# ============================================================

def detect_process(data_url):
    header, encoded = data_url.split(",", 1)
    img_data = b64decode(encoded)
    pil = PILImage.open(BytesIO(img_data)).convert("RGB")

    img = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    print(f"🟦 Wajah terdeteksi: {len(faces)}")

    for (x, y, w, h) in faces:
        roi = gray[y:y+h, x:x+w]

        try:
            label, conf = recognizer.predict(roi)
            print(f"🔍 Recognize → ID:{label}, Confidence:{int(conf)}")
        except:
            print("❌ Unknown")

# ============================================================
#                  FUNGSI SIMPAN WAJAH (TO DRIVE)
# ============================================================

def save_face(data_url):
    header, encoded = data_url.split(",", 1)
    img_data = b64decode(encoded)
    pil = PILImage.open(BytesIO(img_data)).convert("RGB")

    img = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if len(faces) == 0:
        print("❌ Tidak ada wajah ditemukan!")
        return

    x, y, w, h = faces[0]
    face_crop = gray[y:y+h, x:x+w]

    idx = len(os.listdir(DATASET_DIR)) + 1
    filename = f"face_{idx}_{idx}.jpg"

    save_path = os.path.join(DATASET_DIR, filename)
    cv2.imwrite(save_path, face_crop)

    print(f"💾 Wajah disimpan di Google Drive sebagai: {save_path}")

    auto_train()

# ============================================================
#                  FUNGSI RECOGNIZE ONLY
# ============================================================

def recognize_only(data_url):
    header, encoded = data_url.split(",", 1)
    img_data = b64decode(encoded)
    pil = PILImage.open(BytesIO(img_data)).convert("RGB")

    img = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    if len(faces) == 0:
        print("❌ Tidak ada wajah terdeteksi.")
        return

    x, y, w, h = faces[0]
    roi = gray[y:y+h, x:x+w]

    try:
        label, conf = recognizer.predict(roi)
        print(f"🔍 Recognize → ID:{label}, Confidence:{int(conf)}")
    except:
        print("❌ Unknown")

# ============================================================
#                  DAFTARKAN CALLBACK
# ============================================================

output.register_callback("notebook.detect", detect_process)
output.register_callback("notebook.save_face", save_face)
output.register_callback("notebook.recognize_face", recognize_only)

# ============================================================
#                  TAMPILKAN KAMERA + BUTTON
# ============================================================

html_code = """
<div style="text-align:center">
    <h2>Face Detection & Recognition (Google Drive)</h2>
    <video id="video" width="480" height="360" autoplay></video><br><br>

    <button onclick="detek()">🎯 Deteksi</button>
    <button onclick="save()">💾 Simpan Wajah</button>
    <button onclick="rekog()">🔍 Recognize</button>
</div>

<script>
navigator.mediaDevices.getUserMedia({ video: true })
    .then(stream => { document.getElementById("video").srcObject = stream; });

function getFrame() {
    var video = document.getElementById("video");
    var canvas = document.createElement("canvas");
    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;
    canvas.getContext("2d").drawImage(video, 0, 0);
    return canvas.toDataURL("image/jpeg");
}

function detek() {
    google.colab.kernel.invokeFunction("notebook.detect", [getFrame()], {});
}

function save() {
    google.colab.kernel.invokeFunction("notebook.save_face", [getFrame()], {});
}

function rekog() {
    google.colab.kernel.invokeFunction("notebook.recognize_face", [getFrame()], {});
}
</script>
"""

display(HTML(html_code))
print("\n=== Kamera aktif — klik Izinkan jika muncul popup ===")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✔ Folder Drive OK: /content/drive/MyDrive/Face_Project/
⚠ Belum ada model, akan dilatih otomatis setelah menyimpan wajah.



=== Kamera aktif — klik Izinkan jika muncul popup ===
🟦 Wajah terdeteksi: 1
❌ Unknown
💾 Wajah disimpan di Google Drive sebagai: /content/drive/MyDrive/Face_Project/dataset_faces/face_1_1.jpg
✅ Training otomatis selesai & model disimpan ke Google Drive.
🟦 Wajah terdeteksi: 1
🔍 Recognize → ID:1, Confidence:36
💾 Wajah disimpan di Google Drive sebagai: /content/drive/MyDrive/Face_Project/dataset_faces/face_2_2.jpg
✅ Training otomatis selesai & model disimpan ke Google Drive.


In [17]:
%%writefile face_detection_webcam.py
# [Salin SELURUH kode di atas TANPA baris !pip dan !wget]

Writing face_detection_webcam.py


In [21]:
from google.colab import files
import zipfile
import os

zip_name = "dataset_faces.zip"

with zipfile.ZipFile(zip_name, 'w') as z:
    for f in os.listdir():
        if f.startswith('face_') and f.endswith('.jpg'):
            z.write(f)

files.download(zip_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>